### Imports

In [1]:
import sys
import os
import pandas as pd
import plotly.express as px
sys.path.append('..')
from definitions import OUTPUTS_DIR

### Load the results

In [2]:
def load(root : str) -> pd.DataFrame:

    datasets = os.listdir(root)

    data = []

    for dataset in datasets:

        experiments = os.listdir(os.path.join(root, dataset))

        for experiment in experiments:

            if not os.path.isdir(os.path.join(root, dataset, experiment)):
                continue

            path = os.path.join(root, dataset, experiment, 'metrics.csv')

            if not os.path.exists(path):
                continue

            df = pd.read_csv(path)
            df['experiment'] = experiment
            df['dataset'] = dataset

            data.append(df)

    return pd.concat(data)

In [3]:
all = load(OUTPUTS_DIR)
all['loss'] = all['experiment'].apply(lambda x: x.split('_')[0])
all['layer'] = all['experiment'].apply(lambda x: x.split('_')[1])
all['epoch'] = all['epoch'] + 10

In [4]:
all.head()

,filename,iou,dice,time,epoch,experiment,dataset,loss,layer
0,('/home/abdelnour/Documents/ESI/5eme_anne/GNN-...,0.897419,0.945936,1.381423,10,cc_gcn,emd6,cc,gcn
1,('/home/abdelnour/Documents/ESI/5eme_anne/GNN-...,0.897068,0.945741,1.381423,20,cc_gcn,emd6,cc,gcn
2,('/home/abdelnour/Documents/ESI/5eme_anne/GNN-...,0.897068,0.945741,1.381423,30,cc_gcn,emd6,cc,gcn
3,('/home/abdelnour/Documents/ESI/5eme_anne/GNN-...,0.897068,0.945741,1.381423,40,cc_gcn,emd6,cc,gcn
4,('/home/abdelnour/Documents/ESI/5eme_anne/GNN-...,0.897462,0.945960,1.381423,50,cc_gcn,emd6,cc,gcn


### ISIC

In [5]:
results = all[all['dataset'] == 'isic2016']

- IoU Metric by epoch

In [6]:
data = results.groupby(['loss','layer','epoch']).mean(numeric_only=True).reset_index()

fig = px.line(
    data,
    x='epoch',
    y='iou',
    color='layer',       # Corresponds to 'hue' in Seaborn
    line_dash='loss',    # Corresponds to 'style' in Seaborn
    markers=True         # Adds markers for data points
)

# Update legend position
fig.update_layout(
    width=800,  # Set figure width (in pixels)
    height=600, # Set figure height (in pixels)
    legend=dict(
        x=1,  # Adjust the x position
        y=1,  # Adjust the y position
        xanchor='left',
        yanchor='top'
    ),
    title="IoU vs. Epoch",
    xaxis_title="Epoch",
    yaxis_title="IoU",
)

- Print best epochs

In [7]:
for loss_layer in (data['layer']+'-'+data['loss']).unique():
    data_loss_layer = data[data['layer']+'-'+data['loss'] == loss_layer]
    best_epoch = data_loss_layer.iloc[data_loss_layer['iou'].argmax()].epoch
    print(f'Best epoch for {loss_layer}: {best_epoch}')

Best epoch for gat-cc: 80
Best epoch for gcn-cc: 20
Best epoch for arma-dmon: 80
Best epoch for gat-dmon: 90
Best epoch for gcn-dmon: 100
Best epoch for arma-ncut: 30
Best epoch for gat-ncut: 100
Best epoch for gcn-ncut: 90


- Comparing models according to IoU

In [8]:
data_max_by_epoch = data.groupby(['loss','layer']).max().reset_index()

# Create the bar plot
fig = px.bar(
    data_max_by_epoch,
    x='loss',
    y='iou',
    color='layer',  # Corresponds to 'hue' in Seaborn
    barmode='group' # Grouped bars for each layer
)

# Update layout for figure size, legend, and titles
fig.update_layout(
    width=800,  # Set figure width
    height=600, # Set figure height
    legend=dict(
        x=1,        # Adjust x position of the legend
        y=1,        # Adjust y position of the legend
        xanchor='left',
        yanchor='top'
    ),
    title="Maximum IoU by Loss and Layer",
    xaxis_title="Loss",
    yaxis_title="IoU"
)

# Add bar labels
fig.update_traces(
    texttemplate='%{y:.2f}',  # Format labels with two decimal points
    textposition='outside'    # Position labels outside the bars
)

- Comparing metrics according to DICE

In [9]:
fig = px.bar(
    data_max_by_epoch,
    x='loss',
    y='dice',
    color='layer',  # Corresponds to 'hue' in Seaborn
    barmode='group' # Grouped bars for each layer
)

# Update layout for figure size, legend position, and titles
fig.update_layout(
    width=800,  # Set figure width
    height=600, # Set figure height
    legend=dict(
        x=1,        # Adjust x position of the legend
        y=1,        # Adjust y position of the legend
        xanchor='left',
        yanchor='top'
    ),
    title="Dice Score by Loss and Layer",
    xaxis_title="Loss",
    yaxis_title="Dice Score"
)

# Add bar labels
fig.update_traces(
    texttemplate='%{y:.2f}',  # Format labels with two decimal points
    textposition='outside'    # Position labels outside the bars
)

### EMD6

In [10]:
results = all[all['dataset'] == 'emd6']

- IoU Vs Epoch

In [11]:
data = results.groupby(['loss','layer','epoch']).mean(numeric_only=True).reset_index()

fig = px.line(
    data,
    x='epoch',
    y='iou',
    color='layer',       # Corresponds to 'hue' in Seaborn
    line_dash='loss',    # Corresponds to 'style' in Seaborn
    markers=True         # Adds markers for data points
)

# Update legend position
fig.update_layout(
    width=800,  # Set figure width (in pixels)
    height=600, # Set figure height (in pixels)
    legend=dict(
        x=1,  # Adjust the x position
        y=1,  # Adjust the y position
        xanchor='left',
        yanchor='top'
    ),
    title="IoU vs. Epoch",
    xaxis_title="Epoch",
    yaxis_title="IoU",
)

- Print best epoch

In [12]:
for loss_layer in (data['layer']+'-'+data['loss']).unique():
    data_loss_layer = data[data['layer']+'-'+data['loss'] == loss_layer]
    best_epoch = data_loss_layer.iloc[data_loss_layer['iou'].argmax()].epoch
    print(f'Best epoch for {loss_layer}: {best_epoch}')

Best epoch for gat-cc: 100
Best epoch for gcn-cc: 40
Best epoch for arma-dmon: 80
Best epoch for gat-dmon: 90
Best epoch for gcn-dmon: 70
Best epoch for arma-ncut: 30
Best epoch for gat-ncut: 100
Best epoch for gcn-ncut: 50


- Comparing models according to IoU

In [13]:
data_max_by_epoch = data.groupby(['loss','layer']).max().reset_index()

# Create the bar plot
fig = px.bar(
    data_max_by_epoch,
    x='loss',
    y='iou',
    color='layer',  # Corresponds to 'hue' in Seaborn
    barmode='group' # Grouped bars for each layer
)

# Update layout for figure size, legend, and titles
fig.update_layout(
    width=800,  # Set figure width
    height=600, # Set figure height
    legend=dict(
        x=1,        # Adjust x position of the legend
        y=1,        # Adjust y position of the legend
        xanchor='left',
        yanchor='top'
    ),
    title="Maximum IoU by Loss and Layer",
    xaxis_title="Loss",
    yaxis_title="IoU"
)

# Add bar labels
fig.update_traces(
    texttemplate='%{y:.2f}',  # Format labels with two decimal points
    textposition='outside'    # Position labels outside the bars
)

- Comparing models according to dice

In [14]:
fig = px.bar(
    data_max_by_epoch,
    x='loss',
    y='dice',
    color='layer',  # Corresponds to 'hue' in Seaborn
    barmode='group' # Grouped bars for each layer
)

# Update layout for figure size, legend position, and titles
fig.update_layout(
    width=800,  # Set figure width
    height=600, # Set figure height
    legend=dict(
        x=1,        # Adjust x position of the legend
        y=1,        # Adjust y position of the legend
        xanchor='left',
        yanchor='top'
    ),
    title="Dice Score by Loss and Layer",
    xaxis_title="Loss",
    yaxis_title="Dice Score"
)

# Add bar labels
fig.update_traces(
    texttemplate='%{y:.2f}',  # Format labels with two decimal points
    textposition='outside'    # Position labels outside the bars
)

### Comparing different layers complexity

In [15]:
data = results[['time','layer']].groupby(['layer']).mean().reset_index()

# Create a bar plot
fig = px.bar(
    data,
    x='layer', 
    y='time', 
    text='time',  # Use the 'time' column for bar labels
    title="Average Time by Layer",
    labels={'time': 'Average Time', 'layer': 'Layer'}
)

# Customize bar labels
fig.update_traces(
    texttemplate='%{text:.2f}',  # Format labels to 2 decimal places
    textposition='outside'      # Position labels at the edge
)

# Customize layout
fig.update_layout(
    width=800,  # Set figure width
    height=600, # Set figure height
    xaxis_title="Layer",
    yaxis_title="Average Time",
    title_x=0.5,  # Center the title
    showlegend=False  # Disable legend as it is not needed for this plot
)